Install the tweepy library

In [1]:
# !pip install tweepy

Import required libraries

In [2]:
import os
import tweepy as tw
import pandas as pd

Create Tweeter Consumer Keys and Access Tokens 

In [3]:
consumer_key= 'N/FIO7bIgwQpdCZ7p7rgewKXss1'
consumer_secret= '2/QegNrIpcGCW6nCWFTL5JEsrDCmYZpQHJ8SdIT7V0ql5cQgFwV1'
access_token= '3/049227557-36ln3qak9xbxHvgNos45cf3uScGxuUdaDAR4vqs1'
access_token_secret= 'p/Z4Y0p31graKrAYAy41mYztvIrYFzof7awiTDWQr2ui9u1'

Initialize AuthHandler

In [4]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

Define search words and date

In [5]:
search_words = "#Covid"
date_since = "2018-11-16"

In [6]:
tweets = tw.Cursor(api.search, q=search_words, lang="en", since=date_since).items(5)

In [7]:
tweets

Iterate through tweets using for loop

In [8]:
for tweet in tweets:
    print(tweet.text)

RT @MalikAliKhatta4: In technical and vocational education and training systems, vulnerabilities including low levels of digitalization and…
RT @AffairsCare: I feel that if you are blessed, or lucky enough, to be doing well, you should help others.
- Laurell K. Hamilton
#projectb…
Asked my mom “I mean, can you even imagine spending nine months basically chained to your child?” She said “uh, yes… https://t.co/9PtR1Qd6Tg
45% of Americans are high risk of dying if they get #COVID "Underlying conditions...pivotal in determining who surv… https://t.co/nB1lg5MnHv
RT @and_kell: Saying "I believe in #science" or some form of that statement has become the most overused phrase in 2020 with #covid. 
What…


Retrieve tweets as a collection of list

In [9]:
[tweet.text for tweet in tweets]

[]

Search Tweets without Retweets

In [10]:
new_search = search_words + " -filter:retweets"

In [11]:
new_search

'#Covid -filter:retweets'

In [12]:
tweets = tw.Cursor(api.search, q=new_search, lang="en",since=date_since).items(5)

In [13]:
[tweet.text for tweet in tweets]

['Asked my mom “I mean, can you even imagine spending nine months basically chained to your child?” She said “uh, yes… https://t.co/9PtR1Qd6Tg',
 '45% of Americans are high risk of dying if they get #COVID "Underlying conditions...pivotal in determining who surv… https://t.co/nB1lg5MnHv',
 'The disruptions caused by #COVID-19 to everyday \nlife meant that as many as 40 million children \nworldwide have mis… https://t.co/D2lpRB5udV',
 'Wow. I had a dream last night I was at Ole Miss to watch a @OleMissFB Game and they were playing western Michigan (… https://t.co/m3XNJwW8EW',
 '20 minutes from start to finish! Excellent organisation of mass #Covid testing this morning at St Marx. Result via… https://t.co/VngbMVg6Se']

Get username and location

In [14]:
tweets = tw.Cursor(api.search,q=new_search, lang="en", since=date_since).items(5)

In [15]:
users_locs = [[tweet.user.screen_name,tweet.user.location,tweet.text] for tweet in tweets]

In [16]:
users_locs

[['John62256577',
  '',
  'Asked my mom “I mean, can you even imagine spending nine months basically chained to your child?” She said “uh, yes… https://t.co/9PtR1Qd6Tg'],
 ['EchoingHigher',
  'United States',
  '45% of Americans are high risk of dying if they get #COVID "Underlying conditions...pivotal in determining who surv… https://t.co/nB1lg5MnHv'],
 ['IramShahidButt',
  '',
  'The disruptions caused by #COVID-19 to everyday \nlife meant that as many as 40 million children \nworldwide have mis… https://t.co/D2lpRB5udV'],
 ['mbrudnak',
  'Milwaukee, WI',
  'Wow. I had a dream last night I was at Ole Miss to watch a @OleMissFB Game and they were playing western Michigan (… https://t.co/m3XNJwW8EW'],
 ['AradBenkoe',
  '',
  '20 minutes from start to finish! Excellent organisation of mass #Covid testing this morning at St Marx. Result via… https://t.co/VngbMVg6Se']]

Create a Pandas Dataframe From A List of Tweet Data

In [17]:
tweet_df = pd.DataFrame(data=users_locs, columns=['user', "location","tweet"])

In [18]:
tweet_df

,user,location,tweet
0,John62256577,,"Asked my mom “I mean, can you even imagine spe..."
1,EchoingHigher,United States,45% of Americans are high risk of dying if the...
2,IramShahidButt,,The disruptions caused by #COVID-19 to everyda...
3,mbrudnak,"Milwaukee, WI",Wow. I had a dream last night I was at Ole Mis...
4,AradBenkoe,,20 minutes from start to finish! Excellent org...


https://www.earthdatascience.org/courses/use-data-open-source-python/intro-to-apis/social-media-text-mining-python/